## 16장 텍스트 마이닝

### (1) 웹 스크래핑

In [ ]:
#1. 모듈 및 함수 불러오기
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re

#2. 리뷰와 평점을 담을 빈 리스트 생성
score = []
review = []

#3. 리뷰 스크래핑
for i in range(1, 1001) :
    #3-1. HTML 소스코드 가져오기
    base_url="https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=161967&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page="+str(i)
    html = requests.get(base_url)

    #3-2. HTML 파싱
    soup = bs(html.text, 'html.parser')

    #3-3. 평점 및 리뷰 텍스트 추출
    for j in range(10):
        score.append(soup.select('div.star_score > em')[j].text)
        review.append(soup.find_all('span', {'id': re.compile('_filtered_ment_\d')})[j].text.strip())
        
#4.하나의 데이터프레임으로 저장
df = pd.DataFrame(list(zip(review, score)), columns=['review', 'score'])
df.head()

In [ ]:
df['score'].value_counts()

### (2) 빈도 분석

- 데이터 전처리

In [ ]:
#1. 모듈 및 함수 불러오기
from konlpy.tag import Okt

#2. 문자열 변환
string= ",".join(review)

#3. 명사 추출
okt = Okt( )
nouns=okt.nouns(string)

#4. 두 글자 이상 단어 추출
word_list =[x for x in nouns if len(x) >= 2]

#5. 불용어 제거
stopwords=["영화", "그냥", "정말", "진짜"]
word_list=[i for i in word_list if i not in stopwords]

print(word_list)

- 빈도 분석 수행

In [ ]:
from collections import Counter

count = Counter(word_list)
print(count)


- 워드클라우드 생성

In [ ]:
#1. 라이브러리 설치 및 모듈/함수 불러오기
!pip install wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt


#2. 워드클라우드 생성
font_path = '/usr/share/fonts/truetype/nanum/malgun.ttf'
wc = WordCloud(width = 800, height = 800, font_path = font_path, background_color='white')

#3. 시각화
plt.figure(figsize=(10, 10))
plt.imshow(wc.generate_from_frequencies(count))
plt.axis("off")

### (3) 버즈 분석

- 오픈 API를 호출하여 데이터 불러오기

In [ ]:
#1. 라이브러리 불러오기
import json
import urllib

#2. 요청 정보 입력
request = urllib.request.Request("https://openapi.naver.com/v1/datalab/search")
request.add_header("X-Naver-Client-Id", "vppPR7VLu9XbWzzbCuDL")
request.add_header("X-Naver-Client-Secret","O758KnQqQm")
request.add_header("Content-Type","application/json") 

#3. 요청 본문 생성
body_dict = {"startDate": "2019-01-01",
            "endDate" : "2019-12-20",
            "timeUnit": "month",
            "keywordGroups":[{"groupName": "기생충","keywords":["기생충"]}]}
body = json.dumps(body_dict)
body 


In [ ]:
#1. 서버에 정보 요청
response = urllib.request.urlopen(request, data=body.encode("utf-8"))

#2. 응답 상태 코드 가져오기
rescode = response.getcode( )

#3. 값이 200인 경우에만 데이터 추출
if(rescode==200):
    scraped = response.read( )
else:
    print("Error Code:" + rescode)

#4. json 데이터 타입 변환
result = json.loads(scraped)
result

- 버즈 그래프 출력

In [ ]:
#1. 모듈 및 함수 임포트
import pandas as pd
import matplotlib.pyplot as plt

#2. 데이터 프레임 변환
data = pd.DataFrame(result["results"][0]["data"]).set_index('period')

#3. 그래프 생성
data.plot(color = 'g', figsize = (20,10))
plt.legend(fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.show()


### (4) 토픽 모델링

- 데이터 전처리

In [ ]:
#1. 모듈 및 함수 불러오기
from sklearn.feature_extraction.text import TfidfVectorizer

#2. 문서 단어 행렬 변환
stopword = ['영화', '하지만', '근데', '대한', '이게', '없는', '영화가', '영화는', '영화를', 'ㅋㅋ', '내내', '봤습니다', '보고', '보는', '그냥', '많이', '내가',  '그리고', '진짜', '정말', '너무', '나는', '있는', '가장', 'ㅎㅎ']
tv= TfidfVectorizer(max_df=.15, ngram_range=(1,4), min_df=2, stop_words = stopword)
vect = tv.fit_transform(review)

print("문서 단어 행렬 변환 결과: \n", vect.toarray())

- 토픽 모델링 수행

In [ ]:
#1. 모듈 및 함수 불러오기
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation

#2. LDA 모형 생성 및 변환
model = LatentDirichletAllocation(n_components=5, learning_method="batch", max_iter=25, random_state=0)
model.fit_transform(vect)

#3. 토픽별 주요 단어 출력
for topic_index, topic in enumerate(model.components_):
    print('Topic #',topic_index+1) 
    topic_index = topic.argsort()[::-1]
    feature_names = ' '.join([tv.get_feature_names()[i] for i in topic_index[:10]])